In [2]:
import arcpy
import pandas as pd
import os
import numpy as np 
from time import time

arcpy.env.overwriteOutput = True

In [3]:
#set source for hexagons
hexagons_state = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_Hexagons\hex_policies_claims_all.shp"
#set source for huc selection
huc12 = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\wbdhuc12_50c_Indiana.shp"



In [4]:
#set workspace for scratch files
workspace = arcpy.env.workspace = r"C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\"


In [6]:
#clip state hexagons to just those that intersect with HUC12 selection

hexagons_intersect = arcpy.SelectLayerByLocation_management(hexagons_state, "INTERSECT", huc12) 

hexagons_clip = arcpy.CopyFeatures_management(hexagons_intersect, "hexagons_intersect.shp")

#add area geometry attributes
hexagons = arcpy.AddGeometryAttributes_management(hexagons_clip, "AREA_GEODESIC", '', "SQUARE_KILOMETERS")

huc12_addfield = arcpy.AddField_management(huc12, "area_full", "DOUBLE")

huc12 = arcpy.CalculateGeometryAttributes_management(huc12_addfield, [["area_full","AREA_GEODESIC"]], '', 
                                                     "SQUARE_KILOMETERS")


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

In [7]:
st = time()

#select one hexagon at a time
#search cursor through each row of the hexagons shapefile
with arcpy.da.SearchCursor(hexagons, ["GRID_ID", "AREA_GEO", "ttlbldn_su", "ttlcntn_su", "ttlnsrn_su", "ttlbldn_av",
                                     "ttlcntn_av","ttlnsrn_av", "pbldg_sum", "pcont_sum", 
                                      "ttlblg_sum", "ttlcont_su", "pbldg_avg", "pcont_avg",
                                     "ttlblg_avg", "ttlcont_av"]) as cursor:

    for row in cursor:
        hexagon_id = row[0]

        #print(hexagon_id)

        hexagon_selection = arcpy.SelectLayerByAttribute_management(hexagons, "NEW_SELECTION",
                                                                    f"GRID_ID = '{hexagon_id}'")

        hexagon_copy = arcpy.CopyFeatures_management(hexagon_selection, "hexagon_copy.shp")
        
        hexagon_area = row[1]
        #print(hexagon_area)

        policy_total_building_coverage_sum = row[2]

        policy_total_contents_coverage_sum = row[3]

        policy_total_insurance_premium_sum = row[4]

        policy_total_building_coverage_avg = row[5]

        policy_total_contents_coverage_avg = row[6]

        policy_total_insurance_premium_avg = row[7]

        claims_amt_paid_building_sum = row[8]

        claims_amt_paid_contents_sum = row[9]

        claims_total_building_insurance_coverage_sum = row[10]

        claims_total_contents_insurance_coverage_sum = row[11]

        claims_amt_paid_building_avg = row[12]

        claims_amt_paid_contents_avg = row[13]

        claims_total_building_insurance_coverage_avg = row[14]

        claims_total_contents_insurance_coverage_avg = row[15]






        #clip the subwatersheds to the hexagon selected
        huc12_clip = arcpy.Clip_analysis(huc12, hexagon_copy, 
                                         f"huc12_clip{hexagon_id}.shp")


        huc12_clip_areas = arcpy.AddGeometryAttributes_management(huc12_clip, 
                                                                  "AREA_GEODESIC", "" ,
                                                                  "SQUARE_KILOMETERS")


        

        ws_id_list = []
        fraction_hex_area_list = []
        ws_hex_id_list = []
        hex_area_list = []
        ws_part_area_list = []
        ws_full_area_list = []
        ws_fraction_list = []
        policy_total_building_coverage_sum_list = []
        policy_total_contents_coverage_sum_list = []
        policy_total_insurance_premium_sum_list = []
        policy_total_building_coverage_avg_list = []
        policy_total_contents_coverage_avg_list = []
        policy_total_insurance_premium_avg_list = []
        claims_amt_paid_building_sum_list = []
        claims_amt_paid_contents_sum_list = []
        claims_total_building_insurance_coverage_sum_list = []
        claims_total_contents_insurance_coverage_sum_list = []
        claims_amt_paid_building_avg_list = []
        claims_amt_paid_contents_avg_list = []
        claims_total_building_insurance_coverage_avg_list = []
        claims_total_contents_insurance_coverage_avg_list = []






        with arcpy.da.SearchCursor(huc12_clip_areas, ["HUC12", "AREA_GEO", "area_full"]) as swsCursor:


            for subwatershed in swsCursor:
                ws_id = subwatershed[0]

                ws_id_list.append(ws_id)
                #print(ws_id)

                ws_area = subwatershed[1]
                ws_part_area_list.append(ws_area)

                fraction_hex_area = ws_area / hexagon_area 

                fraction_hex_area_list.append(fraction_hex_area)
                #print(fraction_hex_area)

                ws_hex_id_list.append(hexagon_id)


                hex_area_list.append(hexagon_area)

                ws_full_area = subwatershed[2]
                ws_full_area_list.append(ws_full_area)

                ws_fraction = ws_area / ws_full_area
                ws_fraction_list.append(ws_fraction)


                #take percentage of hexagon area and multiply by all of the policy and claims data

                a = policy_total_building_coverage_sum * fraction_hex_area
                policy_total_building_coverage_sum_list.append(a)

                b = policy_total_contents_coverage_sum * fraction_hex_area
                policy_total_contents_coverage_sum_list.append(b)

                c = policy_total_insurance_premium_sum * fraction_hex_area
                policy_total_insurance_premium_sum_list.append(c)

                d = policy_total_building_coverage_avg * fraction_hex_area
                policy_total_building_coverage_avg_list.append(d)

                e = policy_total_contents_coverage_avg * fraction_hex_area
                policy_total_contents_coverage_avg_list.append(e)

                f = policy_total_insurance_premium_avg * fraction_hex_area
                policy_total_insurance_premium_avg_list.append(f)

                g = claims_amt_paid_building_sum * fraction_hex_area
                claims_amt_paid_building_sum_list.append(g)

                h = claims_amt_paid_contents_sum * fraction_hex_area
                claims_amt_paid_contents_sum_list.append(h)

                i = claims_total_building_insurance_coverage_sum * fraction_hex_area
                claims_total_building_insurance_coverage_sum_list.append(i)

                j = claims_total_contents_insurance_coverage_sum * fraction_hex_area
                claims_total_contents_insurance_coverage_sum_list.append(j)

                k = claims_amt_paid_building_avg * fraction_hex_area
                claims_amt_paid_building_avg_list.append(k)

                l = claims_amt_paid_contents_avg * fraction_hex_area
                claims_amt_paid_contents_avg_list.append(l)

                m = claims_total_building_insurance_coverage_avg * fraction_hex_area
                claims_total_building_insurance_coverage_avg_list.append(m)

                n = claims_total_contents_insurance_coverage_avg * fraction_hex_area
                claims_total_contents_insurance_coverage_avg_list.append(n)











        outputs = {"fraction_hex_area": fraction_hex_area_list,
                  "ws_id": ws_id_list,
                  'hex_id': ws_hex_id_list,
                   'hex_area': hex_area_list,
                   'ws_part_area':  ws_part_area_list,
                    'ws_full_area': ws_full_area_list,
                   'ws_fraction' : ws_fraction_list,
                   "policy_total_building_coverage_sum": policy_total_building_coverage_sum_list,
        "policy_total_contents_coverage_sum": policy_total_contents_coverage_sum_list,
       "policy_total_insurance_premium_sum": policy_total_insurance_premium_sum_list,
        "policy_total_building_coverage_avg": policy_total_building_coverage_avg_list,
        "policy_total_contents_coverage_avg": policy_total_contents_coverage_avg_list,
        "policy_total_insurance_premium_avg": policy_total_insurance_premium_avg_list,
        "claims_amt_paid_building_sum": claims_amt_paid_building_sum_list,
        "claims_amt_paid_contents_sum": claims_amt_paid_contents_sum_list,
        "claims_total_building_insurance_coverage_sum": claims_total_building_insurance_coverage_sum_list,
        "claims_total_contents_insurance_coverage_sum": claims_total_contents_insurance_coverage_sum_list,
        "claims_amt_paid_building_avg": claims_amt_paid_building_avg_list,
        "claims_amt_paid_contents_avg": claims_amt_paid_contents_avg_list,
        "claims_total_building_insurance_coverage_avg": claims_total_building_insurance_coverage_avg_list,
        "claims_total_contents_insurance_coverage_avg": claims_total_contents_insurance_coverage_avg_list
                  }



        outputs_df = pd.DataFrame(outputs, columns = ['fraction_hex_area',
                                                     'ws_id',
                                                     'hex_id',
                                                      'hex_area',
                                                      'ws_part_area',
                                                      'ws_full_area',
                                                      'ws_fraction',
                                                      "policy_total_building_coverage_sum",
                                                       "policy_total_contents_coverage_sum",
                                                      "policy_total_insurance_premium_sum",
                                                      "policy_total_building_coverage_avg",
                                                      "policy_total_contents_coverage_avg",
                                                      "policy_total_insurance_premium_avg",
                                                      "claims_amt_paid_building_sum",
                                                       "claims_amt_paid_contents_sum",
                                                      "claims_total_building_insurance_coverage_sum",
                                                      "claims_total_contents_insurance_coverage_sum",
                                                      "claims_amt_paid_building_avg",
                                                      "claims_amt_paid_contents_avg",
                                                      "claims_total_building_insurance_coverage_avg",
                                                      "claims_total_contents_insurance_coverage_avg"                                            
                                                     ])

        outputs_df.to_excel(workspace + str(hexagon_id) + ".xlsx")

        print("excel created")

    print(round(((time()-st)/60), 2) , 'minutes to process.')    

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

excel created


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

...

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

excel created


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

excel created
731.62 minutes to process.


In [8]:
paths = []
for root, dirs, files in os.walk(workspace):
    for file in files:
        if file.endswith(".xlsx"):
             #print(os.path.join(root, file))
             s = os.path.join(root, file)
             print(s)
             paths.append(s)

all_data = pd.DataFrame()
for f in paths:
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)
    
    
all_data.to_excel(workspace + "all_hexagons.xlsx")

C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-13.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-14.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-15.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-16.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-21.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-22.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-23.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-24.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-25.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-27.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-28.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-29.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-3.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-4.xlsx
C:\Users\smcavoy\Documents\1_Indiana_Hexagons_Working\\AA-5.xlsx
C:\Users\smca

In [10]:
df_allhex = pd.read_excel(r"C:\Users\smcavoy\Documents\all_hexagons_50c.xlsx")

df_allhex.head()


,Unnamed: 0,Unnamed: 0.1,fraction_hex_area,ws_id,hex_id,hex_area,ws_part_area,ws_full_area,ws_fraction,policy_total_building_coverage_sum,...,policy_total_contents_coverage_avg,policy_total_insurance_premium_avg,claims_amt_paid_building_sum,claims_amt_paid_contents_sum,claims_total_building_insurance_coverage_sum,claims_total_contents_insurance_coverage_sum,claims_amt_paid_building_avg,claims_amt_paid_contents_avg,claims_total_building_insurance_coverage_avg,claims_total_contents_insurance_coverage_avg
0,1170,4,0.022706,40400010301,K-4,120.197160,2.729150,42.233962,0.064620,1.561463e+06,...,1388.849968,20.292242,2613.892865,1388.456525,18366.571885,6711.779762,186.706633,99.175466,1311.897992,479.412840
1,1255,3,0.085918,40400010301,L-4,120.085418,10.317491,42.233962,0.244294,5.181762e+06,...,4725.634773,65.853742,19783.493751,5440.185930,178125.054802,64352.530410,732.721991,201.488368,6597.224252,2383.427052
2,1262,3,0.242603,40400010301,L-5,120.308762,29.187321,42.233962,0.691086,3.359912e+06,...,15597.305372,106.113552,5308.464352,4852.069034,13221.888118,4852.069034,5308.464352,4852.069034,13221.888118,4852.069034
3,1172,6,0.372916,40400010302,K-4,120.197160,44.823443,51.587189,0.868887,2.564540e+07,...,22810.409005,333.278865,42930.458087,22803.947108,301651.745236,110233.966982,3066.461292,1628.853365,21546.553231,7873.854784
4,1257,5,0.056324,40400010302,L-4,120.085418,6.763746,51.587189,0.131113,3.396962e+06,...,3097.942751,43.171157,12969.290686,3566.374757,116771.872719,42187.021352,480.344099,132.087954,4324.884175,1562.482272


In [14]:
df_allhex["policy_total_building_coverage_avgw"] = df_allhex["ws_fraction"] * df_allhex["policy_total_building_coverage_avg"]

df_allhex["policy_total_contents_coverage_avgw"] = df_allhex["ws_fraction"] * df_allhex["policy_total_contents_coverage_avg"]

df_allhex["policy_total_insurance_premium_avgw"] = df_allhex["ws_fraction"] * df_allhex["policy_total_insurance_premium_avg"]

df_allhex["claims_amt_paid_building_avgw"] = df_allhex["ws_fraction"] * df_allhex["claims_amt_paid_building_avg"]

df_allhex["claims_amt_paid_contents_avgw"] = df_allhex["ws_fraction"] * df_allhex["claims_amt_paid_contents_avg"]

df_allhex["claims_total_building_insurance_coverage_avgw"] = df_allhex["ws_fraction"] * df_allhex["claims_total_building_insurance_coverage_avg"]

df_allhex["claims_total_contents_insurance_coverage_avgw"] = df_allhex["ws_fraction"] * df_allhex["claims_total_contents_insurance_coverage_avg"]



In [17]:
df_allhex_sum = df_allhex[["ws_id", 
                          "ws_fraction", 
                          "policy_total_building_coverage_avgw", 
                          "policy_total_contents_coverage_avgw",
                         "policy_total_insurance_premium_avgw",
                          "claims_amt_paid_building_avgw", 
                          "claims_amt_paid_contents_avgw",
                          "claims_total_building_insurance_coverage_avgw",
                          "claims_total_contents_insurance_coverage_avgw",
                         "policy_total_building_coverage_sum", 
                          "policy_total_contents_coverage_sum", 
                          "policy_total_insurance_premium_sum",
                          "claims_amt_paid_building_sum", 
                          "claims_amt_paid_contents_sum", 
                          "claims_total_building_insurance_coverage_sum",
                          "claims_total_contents_insurance_coverage_sum"]]



In [18]:
df_allhex_sum.head()

,ws_id,ws_fraction,policy_total_building_coverage_avgw,policy_total_contents_coverage_avgw,policy_total_insurance_premium_avgw,claims_amt_paid_building_avgw,claims_amt_paid_contents_avgw,claims_total_building_insurance_coverage_avgw,claims_total_contents_insurance_coverage_avgw,policy_total_building_coverage_sum,policy_total_contents_coverage_sum,policy_total_insurance_premium_sum,claims_amt_paid_building_sum,claims_amt_paid_contents_sum,claims_total_building_insurance_coverage_sum,claims_total_contents_insurance_coverage_sum
0,40400010301,0.064620,281.062471,89.747215,1.311281,12.064946,6.408699,84.774593,30.979565,1.561463e+06,4.985971e+05,7284.914893,2613.892865,1388.456525,18366.571885,6711.779762
1,40400010301,0.244294,3648.045494,1154.442786,16.087654,178.999363,49.222338,1611.660298,582.256204,5.181762e+06,1.639795e+06,22851.248469,19783.493751,5440.185930,178125.054802,64352.530410
2,40400010301,0.691086,28666.543039,10779.087147,73.333643,3668.608038,3353.199397,9137.468356,3353.199397,3.359912e+06,1.263382e+06,8595.197690,5308.464352,4852.069034,13221.888118,4852.069034
3,40400010302,0.868887,62069.507050,19819.669931,289.581703,2664.408633,1415.289662,18721.521964,6841.490781,2.564540e+07,8.188937e+06,119647.112430,42930.458087,22803.947108,301651.745236,110233.966982
4,40400010302,0.131113,1283.531914,406.180285,5.660296,62.979312,17.318436,567.048144,204.861596,3.396962e+06,1.074986e+06,14980.391617,12969.290686,3566.374757,116771.872719,42187.021352


In [20]:
df_new = df_allhex_sum.groupby(["ws_id"]).sum()

df_new.head()

df_new.to_excel(r"C:\Users\smcavoy\Documents\all_hexagons_50c_grouped.xlsx")